In [7]:
import torch.utils.data
import sklearn.metrics
import torch
import pandas
from torch.utils.data import Dataset
import tqdm
import spacy
import numpy as np

from google.colab import files
import io


# nlp = spacy.load('en')
# nlp = spacy.load('en_core_web_lg')


!python -m spacy download en_core_web_lg
### This takes about 5 mins


# python -m spacy link en_core_web_lg en

     |████████████████████████████████| 852.3MB 1.2MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-_ac7byix/wheels/0d/bc/67/e6a9108ab86cd076703af19ad4e0f02f57381ac6583df16249
Successfully built en-core-web-lg

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



In [0]:
nlp = spacy.load('en_core_web_lg')

In [9]:
for token in nlp('hello'):
  print(token)
  print(token.vector)
  print(token.vector.shape)

hello
[ 0.25233    0.10176   -0.67485    0.21117    0.43492    0.16542
  0.48261   -0.81222    0.041321   0.78502   -0.077857  -0.66324
  0.1464    -0.29289   -0.25488    0.019293  -0.20265    0.98232
  0.028312  -0.081276  -0.1214     0.13126   -0.17648    0.13556
 -0.16361   -0.22574    0.055006  -0.20308    0.20718    0.095785
  0.22481    0.21537   -0.32982   -0.12241   -0.40031   -0.079381
 -0.19958   -0.015083  -0.079139  -0.18132    0.20681   -0.36196
 -0.30744   -0.24422   -0.23113    0.09798    0.1463    -0.062738
  0.42934   -0.078038  -0.19627    0.65093   -0.22807   -0.30308
 -0.12483   -0.17568   -0.14651    0.15361   -0.29518    0.15099
 -0.51726   -0.033564  -0.23109   -0.7833     0.018029  -0.15719
  0.02293    0.49639    0.029225   0.05669    0.14616   -0.19195
  0.16244    0.23898    0.36431    0.45263    0.2456     0.23803
  0.31399    0.3487    -0.035791   0.56108   -0.25345    0.051964
 -0.10618   -0.30962    1.0585    -0.42025    0.18216   -0.11256
  0.40576    0.

In [10]:
uploaded = files.upload()

look = pandas.read_csv(io.BytesIO(uploaded['train.tsv']),sep='\t', header=0)
## tsv is tab seperated data

Saving train.tsv to train.tsv


In [11]:
print(look.iloc[0]) ### this is the full entry in the tsv
print(look.iloc[0][0]) ## this is the sentence entry
print(look.iloc[0][1]) ## this is the rating

a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films    apparently reassembled from the cutting room f...
1                                                                                                                                                          0
Name: 0, dtype: object
apparently reassembled from the cutting room floor of any given daytime soap
0


In [13]:
len(look)

6919

In [0]:
### 


class SentimentDataset(Dataset):
  def __init__(self):
#     self.data = pandas\
#     .read_csv('sentiment.tsv',sep='\t', header = 0). groupby('SentenceId').first()
    self.data = pandas.read_csv(io.BytesIO(uploaded['train.tsv']),sep='\t', header=0)
    

    
    self.ordinals = {}
    for sample in range(len(self.data)):
      for token in nlp(self.data.iloc[sample][0], disable=['parser','tagger','ner']):
        if token.text not in self.ordinals:
          self.ordinals[token.text] = len(self.ordinals)


  def __len__(self):
    return len(self.data)
  
  
  def __getitem__(self,idx):

    if type(idx) is torch.Tensor:
      idx = idx.item()

    sample = self.data.iloc[idx]
    token_vectors = []
    
    for token in nlp(sample[0], disable=['parser','tagger','ner']):
      token_vectors.append(token.vector)
      
      
#     torch.tensor(sample[1])) - this is the sentiment
    return (torch.tensor(token_vectors), torch.tensor(len(token_vectors)), torch.tensor(sample[1]))

  


### the group by is the review part of the data, without being interested in th dataset
### when 

sentiment = SentimentDataset()

In [21]:
sentiment.data.iloc[0][1]
sentiment[0]

(tensor([[ 0.0264,  0.0534, -0.1999,  ..., -0.0585, -0.0613, -0.1719],
         [ 0.9621, -0.1497,  0.0861,  ..., -0.0636, -0.6391,  0.1251],
         [ 0.0133, -0.0511, -0.1321,  ...,  0.0640, -0.5301,  0.1901],
         ...,
         [-0.1236,  0.0720,  0.0314,  ..., -0.2018,  0.1018, -0.0100],
         [ 0.3811,  0.1144, -0.0188,  ..., -0.4154, -0.2239, -0.2230],
         [-0.2226, -0.1851,  0.0212,  ..., -0.8747, -0.1474,  0.1266]]),
 tensor(12),
 tensor(0))

In [22]:
### We can look at how many entries are in the on ehot vector - how big is our dictionary
## he called this how many features we have

len (sentiment.ordinals)
len(sentiment)

6919

In [0]:
def collate(batch):
  
  batch.sort(key=lambda x: x[1], reverse = True)
  sequences,lengths, sentiments = zip(*batch)
  sequences = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True)
#   print(sentiments)
  sentiments = torch.stack(sentiments)
  lengths = torch.stack(lengths)
  return sequences, lengths, sentiments
  
  
## this makes all the sentences in the same batch to be the same length

In [30]:
number_for_testing = int(len(sentiment)*0.05)
number_for_training = len(sentiment) - number_for_testing 

train,test = torch.utils.data.random_split(sentiment, [number_for_training,number_for_testing])


# trainloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle = True)
# testloader = torch.utils.data.DataLoader(test, batch_size = 32, shuffle = True)

trainloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle = True, collate_fn = collate)
testloader = torch.utils.data.DataLoader(test, batch_size = 32, shuffle = True, collate_fn = collate)

len(test), len(train)

(345, 6574)

In [31]:
### sentiment.data.Sentiment.unique()
## they had this in their dataset that had the 5 stars - this is just the labels for the 5 stars thing


for batch in trainloader:
  print(batch[0].shape, batch[1].shape,batch[2].shape)
  print(batch[1][0])
  break

torch.Size([32, 45, 300]) torch.Size([32]) torch.Size([32])
tensor(45)


In [47]:
class  Model(torch.nn.Module):
  
  def __init__(self, input_dimensions, size = 128, layers = 1):
    
    super().__init__()
    self.seq = torch.nn.LSTM(input_dimensions, size, layers)
    self.layer_one = torch.nn.Linear(size*layers, size)
    self.activation_one = torch.nn.ReLU()
    self.layer_two = torch.nn.Linear(size,size)
    self.activation_two = torch.nn.ReLU()
    self.shape_outputs = torch.nn.Linear (size,2) ## this is where we put the size of the number of sentiments vector
    ### Even though we only have one sentiment, there is a bug that requires us to put 2 here
    

  def forward(self, inputs, lengths):
    

    number_of_batches = lengths.shape[0]
    packed_inputs = torch.nn.utils.rnn.pack_padded_sequence(inputs, lengths, batch_first = True)
    
    buffer, (hidden, cell) = self.seq(packed_inputs)
    
#     print(hidden.shape)
    buffer = hidden.permute(1,0,2)
#     print(buffer.shape)
    buffer = buffer.contiguous().view(number_of_batches, -1)
#     print(buffer.shape)


    buffer = self.layer_one(buffer)
    buffer = self.activation_one(buffer)
    buffer = self.layer_two(buffer)
    buffer = self.activation_two(buffer)
    buffer = self.shape_outputs(buffer)

    return buffer
  
  
model = Model(sentiment[0][0].shape[1])

print(len(sentiment.ordinals))

13829


In [48]:
optimizer = torch.optim.Adam(model.parameters())
loss_function = torch.nn.CrossEntropyLoss()
model.train()


Model(
  (seq): LSTM(300, 128)
  (layer_one): Linear(in_features=128, out_features=128, bias=True)
  (activation_one): ReLU()
  (layer_two): Linear(in_features=128, out_features=128, bias=True)
  (activation_two): ReLU()
  (shape_outputs): Linear(in_features=128, out_features=2, bias=True)
)

In [51]:
### He did 64 epochs
for epoch in range(1):
  losses = []
  
  for sequences, lengths, sentiments in tqdm.tqdm(trainloader):
    optimizer.zero_grad()
#     print((inputs.t().size()))
#     print(len(sentiments))
    results = model(sequences, lengths)
#     print(outputs.size())
#     print(results[:,0].size())
#     print(torch.Tensor([outputs.numpy()]).t().size())
    outputs1 = torch.Tensor([sentiments.numpy()]).t().long()
    
    loss = loss_function(results, sentiments)
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
  print("Loss {}".format(torch.tensor(losses).mean()))

100%|██████████| 206/206 [00:18<00:00, 10.92it/s]

Loss 0.4774051308631897


In [54]:
#Not lets try to get the classification report


results_buffer = []
actual_buffer = []
with torch.no_grad():
  model.eval()
  for sequences, lengths, sentiments in testloader:
    results = model(sequences, lengths).argmax(dim=1).numpy()
    results_buffer.append(results)
    actual_buffer.append(sentiments)
    
    
    
print(sklearn.metrics.classification_report(np.concatenate(actual_buffer), np.concatenate(results_buffer)))

              precision    recall  f1-score   support

           0       0.77      0.78      0.78       166
           1       0.80      0.79      0.79       179

   micro avg       0.79      0.79      0.79       345
   macro avg       0.79      0.79      0.79       345
weighted avg       0.79      0.79      0.79       345

